# Advanced Joins and Subqueries with SQLAlchemy

## Overview

This topic covers advanced join techniques and sophisticated subquery patterns in SQLAlchemy. You'll learn about different types of joins, complex subquery scenarios, and how to optimize queries using these powerful SQL features.

## Learning Objectives

By the end of this topic, you will be able to:

1. **Master different join types** - INNER, LEFT, RIGHT, OUTER, and CROSS joins
2. **Write complex subqueries** - EXISTS, IN, correlated subqueries, and CTEs
3. **Optimize join performance** - understand join strategies and indexing
4. **Handle complex data relationships** - many-to-many, self-referential, and hierarchical data
5. **Use advanced join patterns** - lateral joins, window functions with joins, and recursive queries

## Prerequisites

- Complete understanding of basic SQLAlchemy ORM
- Familiarity with model relationships
- Knowledge of SQL join concepts
- Understanding of query optimization basics

Let's dive into advanced joins and subqueries!


## 1. Setup and Model Definition

Let's start with a comprehensive library management system that will demonstrate advanced join and subquery techniques:


In [ ]:
# Setup and model definition for advanced joins and subqueries
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float, Index, Date, Enum, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, joinedload, subqueryload, selectinload
from sqlalchemy import func, and_, or_, not_, desc, asc, case, cast, extract, distinct
from sqlalchemy.sql import text
from datetime import datetime, date, timedelta
import enum

# Create database engine
engine = create_engine('sqlite:///advanced_joins_subqueries.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Enums for better data integrity
class BookStatus(enum.Enum):
    AVAILABLE = "available"
    BORROWED = "borrowed"
    RESERVED = "reserved"
    MAINTENANCE = "maintenance"

class MemberStatus(enum.Enum):
    ACTIVE = "active"
    INACTIVE = "inactive"
    SUSPENDED = "suspended"
    EXPIRED = "expired"

class TransactionType(enum.Enum):
    BORROW = "borrow"
    RETURN = "return"
    RENEW = "renew"
    RESERVE = "reserve"

# Association table for many-to-many relationship
book_authors = Table('book_authors', Base.metadata,
    Column('book_id', Integer, ForeignKey('books.id'), primary_key=True),
    Column('author_id', Integer, ForeignKey('authors.id'), primary_key=True)
)

book_categories = Table('book_categories', Base.metadata,
    Column('book_id', Integer, ForeignKey('books.id'), primary_key=True),
    Column('category_id', Integer, ForeignKey('categories.id'), primary_key=True)
)

# Comprehensive library management system models
class Library(Base):
    __tablename__ = 'libraries'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, index=True)
    address = Column(String(300), nullable=False)
    phone = Column(String(20))
    email = Column(String(100), unique=True, nullable=False)
    established_year = Column(Integer)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    books = relationship("Book", back_populates="library", cascade="all, delete-orphan")
    members = relationship("Member", back_populates="library", cascade="all, delete-orphan")
    transactions = relationship("Transaction", back_populates="library", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Library(name='{self.name}')>"

class Author(Base):
    __tablename__ = 'authors'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False, index=True)
    last_name = Column(String(50), nullable=False, index=True)
    birth_date = Column(Date)
    death_date = Column(Date)
    nationality = Column(String(50))
    biography = Column(Text)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    books = relationship("Book", secondary=book_authors, back_populates="authors")
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    def __repr__(self):
        return f"<Author(name='{self.full_name}')>"

class Category(Base):
    __tablename__ = 'categories'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True, index=True)
    description = Column(Text)
    parent_id = Column(Integer, ForeignKey('categories.id'))
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Self-referential relationship
    parent = relationship("Category", remote_side=[id], backref="children")
    books = relationship("Book", secondary=book_categories, back_populates="categories")
    
    def __repr__(self):
        return f"<Category(name='{self.name}')>"

class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(300), nullable=False, index=True)
    isbn = Column(String(20), unique=True, nullable=False, index=True)
    publication_year = Column(Integer, index=True)
    pages = Column(Integer)
    language = Column(String(20), default='English')
    status = Column(Enum(BookStatus), default=BookStatus.AVAILABLE, index=True)
    price = Column(Float)
    edition = Column(String(50))
    description = Column(Text)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    library_id = Column(Integer, ForeignKey('libraries.id'), nullable=False)
    publisher_id = Column(Integer, ForeignKey('publishers.id'))
    
    # Relationships
    library = relationship("Library", back_populates="books")
    publisher = relationship("Publisher", back_populates="books")
    authors = relationship("Author", secondary=book_authors, back_populates="books")
    categories = relationship("Category", secondary=book_categories, back_populates="books")
    transactions = relationship("Transaction", back_populates="book", cascade="all, delete-orphan")
    reviews = relationship("Review", back_populates="book", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Book(title='{self.title}', isbn='{self.isbn}')>"

class Publisher(Base):
    __tablename__ = 'publishers'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, unique=True, index=True)
    address = Column(String(300))
    phone = Column(String(20))
    email = Column(String(100))
    website = Column(String(200))
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    books = relationship("Book", back_populates="publisher", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Publisher(name='{self.name}')>"

class Member(Base):
    __tablename__ = 'members'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False, index=True)
    last_name = Column(String(50), nullable=False, index=True)
    email = Column(String(100), unique=True, nullable=False, index=True)
    phone = Column(String(20))
    address = Column(String(300))
    date_of_birth = Column(Date)
    membership_date = Column(Date, nullable=False, index=True)
    membership_expiry = Column(Date, index=True)
    status = Column(Enum(MemberStatus), default=MemberStatus.ACTIVE, index=True)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    library_id = Column(Integer, ForeignKey('libraries.id'), nullable=False)
    
    # Relationships
    library = relationship("Library", back_populates="members")
    transactions = relationship("Transaction", back_populates="member", cascade="all, delete-orphan")
    reviews = relationship("Review", back_populates="member", cascade="all, delete-orphan")
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    @property
    def age(self):
        if self.date_of_birth:
            today = date.today()
            return today.year - self.date_of_birth.year - ((today.month, today.day) < (self.date_of_birth.month, self.date_of_birth.day))
        return None
    
    def __repr__(self):
        return f"<Member(name='{self.full_name}', email='{self.email}')>"

class Transaction(Base):
    __tablename__ = 'transactions'
    
    id = Column(Integer, primary_key=True)
    transaction_type = Column(Enum(TransactionType), nullable=False, index=True)
    transaction_date = Column(DateTime, nullable=False, index=True)
    due_date = Column(Date)
    return_date = Column(Date)
    fine_amount = Column(Float, default=0.0)
    notes = Column(Text)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    member_id = Column(Integer, ForeignKey('members.id'), nullable=False)
    book_id = Column(Integer, ForeignKey('books.id'), nullable=False)
    library_id = Column(Integer, ForeignKey('libraries.id'), nullable=False)
    
    # Relationships
    member = relationship("Member", back_populates="transactions")
    book = relationship("Book", back_populates="transactions")
    library = relationship("Library", back_populates="transactions")
    
    def __repr__(self):
        return f"<Transaction(type='{self.transaction_type.value}', member_id={self.member_id}, book_id={self.book_id})>"

class Review(Base):
    __tablename__ = 'reviews'
    
    id = Column(Integer, primary_key=True)
    rating = Column(Integer, nullable=False)  # 1-5 stars
    title = Column(String(200))
    comment = Column(Text)
    is_verified = Column(Boolean, default=False)
    helpful_votes = Column(Integer, default=0)
    created_at = Column(DateTime, default=datetime.utcnow, index=True)
    
    # Foreign keys
    member_id = Column(Integer, ForeignKey('members.id'), nullable=False)
    book_id = Column(Integer, ForeignKey('books.id'), nullable=False)
    
    # Relationships
    member = relationship("Member", back_populates="reviews")
    book = relationship("Book", back_populates="reviews")
    
    def __repr__(self):
        return f"<Review(rating={self.rating}, book_id={self.book_id})>"

# Create indexes for performance
Index('idx_books_title_isbn', Book.title, Book.isbn)
Index('idx_members_name_email', Member.first_name, Member.last_name, Member.email)
Index('idx_transactions_member_book', Transaction.member_id, Transaction.book_id)
Index('idx_transactions_date_type', Transaction.transaction_date, Transaction.transaction_type)
Index('idx_reviews_book_rating', Review.book_id, Review.rating)

# Create tables
Base.metadata.create_all(engine)

print("✅ Advanced models created successfully!")
print("Models: Library, Author, Category, Book, Publisher, Member, Transaction, Review")
print("Features: Many-to-many relationships, self-referential relationships, enums, comprehensive indexing")
